In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as gauss
from sklearn.metrics.pairwise import euclidean_distances as l2
%matplotlib inline

In [2]:
# Data preprocessing
faithful_df = pd.read_table('dat/faithful.dat', sep='\s+', header=13, index_col=0)
display(faithful_df)

X = faithful_df.to_numpy()

,eruptions,waiting
1,3.600,79
2,1.800,54
3,3.333,74
4,2.283,62
5,4.533,85
...,...,...
268,4.117,81
269,2.150,46
270,4.417,90
271,1.817,46


In [3]:
# Simple class for plotting the data
class Plotter:
    def __init__(self, save_plot=False):
        self.img_idx = 1
        self.save_plot = save_plot
        
    def plot_clusters(self, X, mu_pred=None, labels=None, sigma_pred=None):
        plt.figure(figsize=(6, 6))
        # plt.rcParams.update({'font.size': 18})
        x_min, x_max = 1.1, 5.5
        y_min, y_max = 40, 100
        
        if mu_pred is not None and sigma_pred is not None:
            x1 = np.linspace(x_min, x_max, 100)
            x2 = np.linspace(y_min, y_max, 100)
            xx1, xx2 = np.meshgrid(x1, x2)
            xx12 = np.hstack((xx1.reshape(-1, 1), xx2.reshape(-1, 1)))
            cmaps = ['Blues', 'Oranges', 'Greens', 'Reds']
            
            for d in range(mu_pred.shape[0]):
                plt.contourf(x1, 
                             x2, 
                             gauss.pdf(xx12, mean=mu_pred[d], cov=sigma_pred[d]).reshape(100, 100), 
                             alpha=0.2, 
                             levels=100,
                             cmap=cmaps[d%mu_pred.shape[0]])
        
        if mu_pred is None or labels is None:
            plt.scatter(X[:, 0], X[:, 1], c='grey', s=10)
        else:
            for d in range(mu_pred.shape[0]):
                plt.scatter(X[labels==d, 0], X[labels==d, 1], label='Cluster %d' % (d+1), s=10)
            plt.scatter(mu_pred[:, 0], mu_pred[:, 1], label='Cluster centers', c='k', s=50)
            plt.legend()
    
        plt.xlabel('Eruption time [min]')
        plt.xlim([x_min, x_max])
        
        plt.ylabel('Waiting time to next eruption [min]')
        plt.ylim([y_min, y_max])
    
        plt.title('Old Faithful Geyser Data')

        if self.save_plot:
            plt.savefig('out/EM_%03d.png' % self.img_idx)
            self.img_idx += 1
        else:
            plt.show()
        
        plt.close()

In [4]:
# Plot the raw data
plotter = Plotter(save_plot=True)
plotter.plot_clusters(X)

### K-means clustering

**using Lloyd's algorithm:**
1. Start with randomly chosen centers.
2. Repeat until convergence:
    - Assign all points to the closest cluster center.
    - Define the new centers as the mean vectors of the current clusters.

In [5]:
def kmeans_cluster(X, K, eps=1e-6, max_iter=100):
    N, D = X.shape
    
    # Initialize means
    mu_pred = X[np.random.choice(N, size=K, replace=False), :]
    
    obj = 1e9
    prev_obj = None
    for _ in range(max_iter):
        # Assign each data point to the closest cluster center
        distances = l2(X, mu_pred, squared=True)
        labels = np.argmin(distances, axis=1)
        
        # Plot
        plotter.plot_clusters(X, mu_pred=mu_pred, labels=labels)
        
        # Calculate the objective
        prev_obj = obj
        obj = 0
        for i in range(N):
            obj += distances[i, labels[i]]
            
        # Exit condition (Lloyd algorithm is guaranteed to decrease in each step)
        if prev_obj - obj < eps:
            break
        
        # Compute new cluster centers
        for i in range(K):
            # Make sure each center has some assigned points
            if np.sum(labels==i) == 0:
                # If not: restart
                return kmeans_cluster(X, K)
            
            mu_pred[i, :] = np.sum(X[labels==i,:], axis=0)/np.sum(labels==i)
    
    return obj, labels, mu_pred

In [6]:
# Execute K-Means clustering with K clusters
K = 2
_, _, mu_pred = kmeans_cluster(X, K)

### Expectation maximization for gaussian processes

For each datum $n \in [1, ..., N]$, the cluster identity $\textbf{c}_n \in \{0, 1\}^K$ (one hot encoded: $\sum_k c_{nk} = 1$) is distributed as follows (with $\pi \in \mathbb{R}_0^+$, $\sum_k \pi_{k} = 1$):

\begin{equation}
p(\textbf{c}_n|\pi) = \prod_{k=1}^{K} \pi_{k}^{c_{nk}} \text{.}
\end{equation}

Each datum $\textbf{x}_n \in \mathbb{R}^D$ is drawn from one of $K$ Gaussian distributions, selected by $\textbf{c}_n$ with probability

\begin{equation}
p(\textbf{x}_n|\textbf{c}_n, \mu, \Sigma) = \prod_{k=1}^{K} \mathcal{N}(\textbf{x}_n; \mu_k, \Sigma_k)^{c_{nk}}
\end{equation}

with $\mu_k \in \mathbb{R}^D$; $\Sigma_k \in \mathbb{R}^{D \times D}, spd$.

**E-Step:**

Applying the Expectation-Step as done in *Pattern Recognition and Machine Learning (Bishop, 2006)* yields the objective, the Expected Completed-Data Log-Likelihood:

\begin{equation}
\mathbb{E}_{p(c|x, \pi, \mu, \Sigma)}[\log{p(x, c|\pi, \mu, \Sigma)}] = \sum_{n=1}^N \sum_{k=1}^K \gamma_{nk} \left(\log{\pi_k + \mathcal{N}(x_n; \mu_k, \Sigma_k)}\right)
\end{equation}

with

\begin{equation}
\gamma_{nk} = \frac{\pi_k \mathcal{N}(x_n; \mu_k, \Sigma_k)}{\sum_{l=1}^K \pi_l \mathcal{N}(x_n; \mu_l, \Sigma_l)} \text{.}
\end{equation}

**M-Step:**

In the Maximization-Step $\pi, \mu, \Sigma$ need to be maximized w.r.t. the objective, which yields:

\begin{equation}
\pi_k^* = \frac{\sum_n \gamma_{nk}}{\sum_{n,l} \gamma_{nl}}
\end{equation}

\begin{equation}
\mu_k^* = \frac{\sum_n \gamma_{nk} x_n}{\sum_{n} \gamma_{nk}}
\end{equation}

\begin{equation}
\Sigma_k^* = \frac{\sum_n \gamma_{nk} (x_n - \mu_k)(x_n - \mu_k)^T}{\sum_{n} \gamma_{nk}}
\end{equation}

**Algorithm:**

1. Initialize $\pi, \mu, \Sigma$ ($\mu$ from K-means clustering)
2. Repeat until convergence:
    - Evaluate $\gamma$ and calculate the Expected-Complete Data Log-Likelihood (E-Step).
    - (Re-)estimate $\pi, \mu, \Sigma$ (M-Step).

In [7]:
def em_gaussian_mixture(X, K, mu_pred, eps=1e-3, max_iter=100):
    N, D = X.shape
    
    # Initialize parameters for EM
    pi_pred = np.ones(shape=(K), dtype=np.float)/K
    sigma_pred = np.zeros(shape=(K, D, D), dtype=np.float)
    for d in range(D):
        sigma_pred[:, d, d] = np.var(X[:, d])/10
    
    obj = 1e9
    prev_obj = None
    for _ in range(max_iter):
        # E-step
        gamma = np.zeros(shape=(N, K))
        for k in range(K):
            gamma[:, k] = gauss.pdf(X, mean=mu_pred[k], cov=sigma_pred[k]) * pi_pred[k]
        gamma = gamma/np.sum(gamma, axis=1, keepdims=True)
        
        # Calculate the objective
        prev_obj = obj
        obj = 0
        for k in range(K):
            obj += np.sum(gamma[:, k] * (np.log(gauss.pdf(X, mean=mu_pred[k], cov=sigma_pred[k])) +\
                                         np.log(pi_pred[k])))
        
        # Calculate cluster labels
        labels = np.argmax(gamma, axis=1)
            
        # Save the plot
        plotter.plot_clusters(X, mu_pred=mu_pred, labels=labels, sigma_pred=sigma_pred)

        # Check for convergence
        if np.abs(obj - prev_obj) < eps:
            break

        # M-step
        pi_pred = np.sum(gamma, axis=0)/np.sum(gamma)
        for k in range(K):
            mu_pred[k, :] = np.sum(gamma[:, k].reshape(-1,1) * X, axis=0) /\
                            np.sum(gamma[:, k], keepdims=True)
            sigma_pred[k, :, :] = np.sum(gamma[:, k].reshape(-1, 1, 1) *\
                                         ((X-mu_pred[k, :]).reshape(N, D, 1) @\
                                          (X-mu_pred[k, :]).reshape(N, 1, D)), axis=0) /\
                                  np.sum(gamma[:, k])
    return obj, labels, pi_pred, mu_pred, sigma_pred

In [8]:
obj, _, pi_pred, mu_pred, sigma_pred = em_gaussian_mixture(X, K, mu_pred)

In [9]:
print('Expected Complete-Data Log-Likelihood:')
print('--------------------------------------')
print('%.3f' % obj)
print()
print('Cluster probabilities:')
print('----------------------')
print(pi_pred)
print()
print('Cluster means:')
print('--------------')
print(mu_pred)
print()
print('Cluster covariance matrices:')
print('----------------------------')
print(sigma_pred)

Expected Complete-Data Log-Likelihood:
--------------------------------------
-1130.959

Cluster probabilities:
----------------------
[0.64412602 0.35587398]

Cluster means:
--------------
[[ 4.28966439 79.96814438]
 [ 2.03639118 54.47854382]]

Cluster covariance matrices:
----------------------------
[[[ 0.16996537  0.94057034]
  [ 0.94057034 36.04577243]]

 [[ 0.06916984  0.43519023]
  [ 0.43519023 33.69743623]]]
